In [111]:
import pandas as pd
from IPython.display import display, HTML

df_zip_to_tract = pd.read_excel("../chosen_data/crosswalk/zip_tract_032022.xlsx")
df_zip_2_cbsa = pd.read_excel("../sample_data/zip_cbsa_122022.xlsx")
df_svi = pd.read_csv("../chosen_data/svi/SVI_2022_US.csv")
df_zillow_d2p = pd.read_csv("../chosen_data/zillow/Metro_mean_doz_pending_uc_sfrcondo_month.csv")
df_zillow_county_crosswalk = pd.read_csv("../chosen_data/zillow/CountyCrossWalk_Zillow.csv")


d:\NASA_Interview\housing_data\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
d:\NASA_Interview\housing_data\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [112]:
import duckdb

duckdb.sql("""
CREATE OR REPLACE TABLE tract_to_zip AS
    SELECT * FROM df_tract_to_zip;
           """)

duckdb.sql("""
CREATE OR REPLACE TABLE zip_2_cbsa AS
    SELECT * FROM df_zip_2_cbsa;
           """)

duckdb.sql("""
CREATE OR REPLACE TABLE svi AS
    SELECT * FROM df_svi;
           """)

duckdb.sql("""
CREATE OR REPLACE TABLE zillow_time2pending AS
    SELECT * FROM df_zillow_d2p;
           """)

duckdb.sql("""
CREATE OR REPLACE TABLE zillow_county_crosswalk AS
    SELECT * FROM df_zillow_county_crosswalk;
           """)

In [115]:
duckdb.sql("SELECT * FROM tract_to_zip LIMIT 5")

┌────────────┬───────┬────────────────────┬─────────────────────┬──────────────────────┬────────────────────┬───────────┬─────────────────────┐
│   TRACT    │  ZIP  │ USPS_ZIP_PREF_CITY │ USPS_ZIP_PREF_STATE │      RES_RATIO       │     BUS_RATIO      │ OTH_RATIO │      TOT_RATIO      │
│   int64    │ int64 │       double       │       double        │        double        │       double       │  double   │       double        │
├────────────┼───────┼────────────────────┼─────────────────────┼──────────────────────┼────────────────────┼───────────┼─────────────────────┤
│ 1001020100 │ 36067 │               NULL │                NULL │                  1.0 │                1.0 │       1.0 │                 1.0 │
│ 1001020200 │ 36067 │               NULL │                NULL │                  1.0 │                1.0 │       1.0 │                 1.0 │
│ 1001020300 │ 36067 │               NULL │                NULL │                  1.0 │                1.0 │       1.0 │               

In [113]:
duckdb.sql("SELECT * FROM zip_2_cbsa LIMIT 10")

┌───────┬───────┬────────────────────┬─────────────────────┬────────────────────────┬───────────┬───────────┬────────────────────────┐
│  ZIP  │ CBSA  │ USPS_ZIP_PREF_CITY │ USPS_ZIP_PREF_STATE │       RES_RATIO        │ BUS_RATIO │ OTH_RATIO │       TOT_RATIO        │
│ int64 │ int64 │      varchar       │       varchar       │         double         │  double   │  double   │         double         │
├───────┼───────┼────────────────────┼─────────────────────┼────────────────────────┼───────────┼───────────┼────────────────────────┤
│   683 │ 41900 │ SAN GERMAN         │ PR                  │     0.9998426188227888 │       1.0 │       1.0 │     0.9998552926705737 │
│   683 │ 32420 │ SAN GERMAN         │ PR                  │ 0.00015738117721120554 │       0.0 │       0.0 │ 0.00014470732942623544 │
│   923 │ 41980 │ SAN JUAN           │ PR                  │                    1.0 │       1.0 │       1.0 │                    1.0 │
│  1010 │ 44140 │ BRIMFIELD          │ MA              

In [114]:
duckdb.sql("SELECT * FROM zillow_county_crosswalk LIMIT 5")

┌────────────┬──────────────┬───────────┬────────────┬──────────────────┬───────────────────────────────────────┬───────────────────────┬──────────────────────┬───────┬──────────┐
│ CountyName │  StateName   │ StateFIPS │ CountyFIPS │ MetroName_Zillow │               CBSAName                │ CountyRegionID_Zillow │ MetroRegionID_Zillow │ FIPS  │ CBSACode │
│  varchar   │   varchar    │   int64   │   int64    │     varchar      │                varchar                │         int64         │        double        │ int64 │  double  │
├────────────┼──────────────┼───────────┼────────────┼──────────────────┼───────────────────────────────────────┼───────────────────────┼──────────────────────┼───────┼──────────┤
│ Pike       │ Pennsylvania │        42 │        103 │ New York, NY     │ New York-Newark-Jersey City, NY-NJ-PA │                   280 │             394913.0 │ 42103 │  35620.0 │
│ Bronx      │ New York     │        36 │          5 │ New York, NY     │ New York-Newark-Jersey Cit

 NEED TO MAKE WORK FOR MULTIPLE DATE RANGES

In [149]:
duckdb.sql("""
CREATE OR REPLACE TABLE zillow_tract_crosswalk AS 
WITH 
    disinct_ziptract AS (SELECT ZIP AS zip, TRACT AS tract FROM tract_to_zip GROUP BY ZIP, TRACT),
    disinct_cbsazip AS (SELECT CBSA AS cbsa, ZIP AS zip, FROM zip_2_cbsa GROUP BY CBSA, ZIP),
    disinct_zillowcbsa AS (SELECT MetroRegionID_Zillow AS zillow_code, CBSACode AS cbsa FROM zillow_county_crosswalk GROUP BY MetroRegionID_Zillow, CBSACode),
    zillow_zip AS (SELECT zillow_code, disinct_zillowcbsa.cbsa, disinct_cbsazip.zip FROM disinct_zillowcbsa LEFT JOIN disinct_cbsazip ON disinct_zillowcbsa.cbsa = disinct_cbsazip.cbsa)
SELECT zillow_code, tract FROM zillow_zip LEFT JOIN disinct_ziptract ON (zillow_zip.zip = disinct_ziptract.zip) GROUP BY zillow_code, tract)
""")

In [150]:
duckdb.sql("SELECT * FROM zillow_tract_crosswalk")

┌─────────────┬─────────────┐
│ zillow_code │    tract    │
│   double    │    int64    │
├─────────────┼─────────────┤
│    394331.0 │ 18151971000 │
│    394325.0 │ 19169000800 │
│    395030.0 │ 27049080700 │
│    394625.0 │ 17095000100 │
│    394735.0 │ 20209044104 │
│    394437.0 │  8043979400 │
│    394484.0 │  8041007601 │
│    394638.0 │ 31079000100 │
│    395054.0 │ 48451001600 │
│    394459.0 │ 51065020300 │
│        ·    │      ·      │
│        ·    │      ·      │
│        ·    │      ·      │
│    395012.0 │ 37183053602 │
│    394549.0 │ 37183053602 │
│    394332.0 │ 26161411900 │
│    395183.0 │  6113010512 │
│    394913.0 │ 36047025800 │
│    394952.0 │  6111008303 │
│    394974.0 │ 34011020200 │
│    394976.0 │  4013319104 │
│    394727.0 │ 42013100900 │
│    394532.0 │ 26163543500 │
├─────────────┴─────────────┤
│ ? rows          2 columns │
└───────────────────────────┘